- 下記を検証する。
    - model endpoint に複数のjsonファイルを入力できるか？
        - 可能
    - 複数のjsonファイルをinference.pyで前処理できるか？
    - inference.py上でlabelencoderを利用できるか？
        - 使用可能。model.tar.gzにlabel_encodersを含めて、圧縮する必要あり。また、predict_fn()内で行う必要あり。
        - decodeはoutput_fn()で行う。
    - 出力を任意のdataframeの形式にできるか？
        - 不可。json形式でしか出力できない。
    - 出力をS3にできるか？
        - 可能。boto3を利用。
        - output_fnで行う。

# モジュールインポート

In [1]:
# SageMaker関連のモジュール
import sagemaker
from sagemaker.session import Session  # SageMakerセッションの生成
from sagemaker.sklearn import SKLearnModel  # Scikit-learnモデル用のモジュール
from sagemaker.async_inference import AsyncInferenceConfig  # 非同期推論設定
from sagemaker.workflow.pipeline import Pipeline  # SageMaker Pipelineの設定
from sagemaker.workflow.steps import ProcessingStep, TrainingStep  # パイプラインのステップ
from sagemaker.processing import ScriptProcessor  # スクリプトプロセッサ
from sagemaker.sklearn.processing import SKLearnProcessor  # Scikit-learn用のプロセッサ
from sagemaker.workflow.pipeline_context import PipelineSession  # パイプラインセッション
from sagemaker.inputs import TrainingInput  # トレーニングデータの入力
from sagemaker.workflow.step_collections import RegisterModel  # モデル登録
from sagemaker.model_metrics import MetricsSource, ModelMetrics  # モデルメトリクス
from sagemaker.workflow.parameters import ParameterString  # パイプラインパラメータ
from sagemaker.workflow.functions import Join  # ステップ間でのパス結合
from sagemaker.model import Model
from sagemaker import get_execution_role
from sagemaker.image_uris import retrieve
from sagemaker.predictor import Predictor
from sagemaker.serializers import JSONSerializer, CSVSerializer
from sagemaker.deserializers import JSONDeserializer, CSVDeserializer
from sagemaker.local import LocalSession

# 必要な標準ライブラリ
import os  # OS操作
import json  # JSON操作
import logging  # ロギング設定
from io import BytesIO  # バイトストリーム
from time import sleep  # 一時停止処理
from uuid import uuid4  # UUID生成
from typing import Final  # 定数宣言用
import tarfile
import importlib
import random
import joblib
from datetime import datetime

# AWS関連モジュール
import boto3  # AWS SDK for Python

# データ分析・機械学習関連
import numpy as np  # 数値計算ライブラリ
import pandas as pd  # データ処理・分析
import lightgbm as lgb  # LightGBMモデル
from sklearn.model_selection import train_test_split  # データ分割
from sklearn.datasets import make_classification  # サンプルデータ生成
from sklearn.preprocessing import LabelEncoder

# ログの設定
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# コメント:
# - 不要な重複を排除し、モジュールを適切にまとめました。
# - import文をカテゴリーごとに整理することで、可読性を向上させています。
# - ロギング設定を追加し、loggerインスタンスを準備済み。

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


/opt/conda/lib/python3.10/site-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


# 事前設定
- モデル構築前の事前準備を行う。
    - Sagemakerの各種設定
    - 必要ディレクトリの作成
    - サンプルデータの作成
      - S3にアップロード

## Sagemaker各種設定

In [2]:
def initialize_sagemaker_settings():
    """
    SageMakerとAWSの事前設定を行う関数

    :return: 設定情報を含む辞書
    """
    # SageMakerクライアントとセッションの初期化
    smr_client: Final = boto3.client('sagemaker-runtime')
    sm_client: Final = boto3.client('sagemaker')
    s3_client: Final = boto3.client('s3')
    session = sagemaker.Session()

    # エンドポイントの状態を確認するためのウェイター設定
    endpoint_inservice_waiter: Final = sm_client.get_waiter('endpoint_in_service')

    # SageMakerの実行ロールとリージョンの取得
    role: Final[str] = sagemaker.get_execution_role()
    region: Final[str] = session.boto_region_name

    # デフォルトのS3バケットとプレフィックスの設定
    bucket: Final[str] = session.default_bucket()
    s3_prefix = 'model_deploy_template_multi_dict'

    # 設定情報のロギング
    logger.info(f"Initialized SageMaker settings with role={role}, region={region}, bucket={bucket}, s3_prefix={s3_prefix}")

    return {
        "smr_client": smr_client,
        "sm_client": sm_client,
        "s3_client": s3_client,
        "endpoint_inservice_waiter": endpoint_inservice_waiter,
        "role": role,
        "region": region,
        "bucket": bucket,
        "session": session,
        "s3_prefix": s3_prefix
    }

# 設定の初期化
sagemaker_settings = initialize_sagemaker_settings()
print(sagemaker_settings["bucket"])

INFO:__main__:Initialized SageMaker settings with role=arn:aws:iam::706711397653:role/service-role/AmazonSageMaker-ExecutionRole-20240825T162290, region=ap-northeast-1, bucket=sagemaker-ap-northeast-1-706711397653, s3_prefix=model_deploy_template_multi_dict


sagemaker-ap-northeast-1-706711397653


## ディレクトリ設定

In [4]:
def create_directory(directory_path: str):
    """
    指定されたディレクトリを作成する関数。
    既に存在する場合は削除して新しく作成します。

    :param directory_path: 作成するディレクトリのパス
    """
    # 既存ディレクトリの削除
    if os.path.exists(directory_path):
        logger.info(f"Directory '{directory_path}' already exists. Removing it.")
        os.rmdir(directory_path)

    # ディレクトリの作成
    os.makedirs(directory_path, exist_ok=True)
    logger.info(f"Directory '{directory_path}' created successfully.")

# 定数としてディレクトリ名を定義
model_dir: Final[str] = 'model'
source_dir: Final[str] = 'source'

# # 必要なディレクトリを作成
# create_directory(model_dir)
# create_directory(source_dir)

## サンプルデータの作成

In [5]:
def generate_sample_data(
    n_samples: int = 100,
    n_features: int = 10,
    n_classes: int = 2,
    n_categorical_features: int = 2,
    n_categories: int = 3,
    random_state: int = 42,
    output_file: str = 'data.csv'
):
    """
    サンプルデータを生成し、CSVファイルに保存する関数。
    数値特徴量とカテゴリ変数を含みます。

    :param n_samples: 生成するサンプル数 (デフォルト: 100)
    :param n_features: 数値特徴量の数 (デフォルト: 10)
    :param n_classes: クラス数 (デフォルト: 2)
    :param n_categorical_features: カテゴリ変数の数 (デフォルト: 2)
    :param n_categories: 各カテゴリ変数のカテゴリ数 (デフォルト: 3)
    :param random_state: 乱数シード (デフォルト: 42)
    :param output_file: 保存するCSVファイル名 (デフォルト: 'data.csv')
    """
    logger.info("Generating sample data...")

    # 数値特徴量を生成
    X, y = make_classification(
        n_samples=n_samples,
        n_features=n_features,
        n_classes=n_classes,
        random_state=random_state
    )
    
    # 数値データをDataFrameに変換
    data = pd.DataFrame(X, columns=[f"feature_{i}" for i in range(n_features)])
    data['label'] = y

    # カテゴリ変数の生成
    np.random.seed(random_state)
    for i in range(n_categorical_features):
        category_values = [f"Category_{j}" for j in range(n_categories)]
        data[f"categorical_feature_{i}"] = np.random.choice(category_values, size=n_samples)
    
    # データの保存
    data.to_csv(output_file, index=False)
    logger.info(f"Sample data saved to {output_file}")

    return data

# サンプルデータの生成例
data = generate_sample_data()
data

INFO:__main__:Generating sample data...
INFO:__main__:Sample data saved to data.csv


,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,label,categorical_feature_0,categorical_feature_1
0,-1.140526,1.359706,0.861991,0.846092,0.606010,-1.556629,1.754794,1.696456,-1.280429,-2.081929,1,Category_2,Category_2
1,-0.078734,-1.329332,0.627375,-1.193006,-0.773010,0.097676,0.497998,0.959271,0.024510,1.451144,1,Category_0,Category_2
2,0.807427,0.730198,-1.285680,0.889484,-1.804882,-0.763259,0.048085,-0.904317,-1.627542,0.259723,0,Category_2,Category_2
3,0.588465,-0.375121,-0.575002,-0.149518,-0.563725,0.412931,0.243687,-0.506943,-0.822220,0.244967,0,Category_2,Category_0
4,1.636312,-1.640607,-1.360456,-0.941163,-1.430141,1.632411,0.130741,-1.435862,-0.440044,1.441273,0,Category_0,Category_2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.531047,0.948005,-1.032233,0.996309,-1.008086,-2.038125,-0.351513,1.676437,-1.870792,0.018418,0,Category_0,Category_1
96,-0.963142,0.556003,0.964233,0.193754,-0.703176,1.149273,1.770801,1.812449,-0.034988,-0.626967,1,Category_0,Category_2
97,0.950573,-2.255540,-0.270624,-1.692005,0.504047,-0.985726,-0.792873,-1.035242,-0.530258,-0.107030,1,Category_2,Category_2
98,-1.016951,0.554484,1.031102,0.175930,0.615936,-1.081063,-0.309546,-1.251114,0.593101,0.326133,1,Category_0,Category_1


## データをS3にアップロード

In [6]:
def upload_to_s3(file_path: str, bucket_name: str, s3_prefix: str, session: Session) -> str:
    """
    ローカルファイルをS3にアップロードする関数。

    :param file_path: アップロードするローカルファイルのパス
    :param bucket_name: アップロード先のS3バケット名
    :param s3_prefix: S3上の保存先プレフィックス
    :param session: SageMakerのセッション
    :return: アップロードされたファイルのS3 URI
    """
    logger.info(f"Uploading {file_path} to S3 bucket '{bucket_name}' with prefix '{s3_prefix}'.")
    
    # S3にデータをアップロード
    s3_uri = session.upload_data(path=file_path,
                                 bucket=bucket_name,
                                 key_prefix=f'{s3_prefix}/data')
    
    logger.info(f"Data has been uploaded to {s3_uri}")
    return s3_uri

# データをS3にアップロード
data_file = "data.csv"
s3_uri = upload_to_s3(file_path=data_file,
                      bucket_name=sagemaker_settings["bucket"],
                      s3_prefix=sagemaker_settings["s3_prefix"],
                      session=sagemaker_settings["session"])

INFO:__main__:Uploading data.csv to S3 bucket 'sagemaker-ap-northeast-1-706711397653' with prefix 'model_deploy_template_multi_dict'.
INFO:__main__:Data has been uploaded to s3://sagemaker-ap-northeast-1-706711397653/model_deploy_template_multi_dict/data/data.csv


# 前処理

## ラベルエンコーディング

In [7]:
def label_encode_categorical_columns(data: pd.DataFrame, encoder_file: str = "encoders.pkl"):
    """
    カテゴリ変数を見つけてラベルエンコーディングし、エンコーダーを保存する関数。
    
    :param data: ラベルエンコーディングするDataFrame
    :param encoder_file: エンコーダーを保存するファイル名 (デフォルト: "encoders.pkl")
    :return: ラベルエンコーディング済みのDataFrameとエンコーダーの辞書
    """
    label_encoders = {}
    
    # カテゴリ変数のカラムを自動検出してラベルエンコーディング
    for column in data.select_dtypes(include=['object']).columns:
        le = LabelEncoder()
        data[column] = le.fit_transform(data[column])
        label_encoders[column] = le  # エンコーダーを辞書に保存
    
    # エンコーダーを指定のファイルに保存
    encoders_path = os.path.join("model/", "encoders.joblib")
    joblib.dump(label_encoders, encoders_path)
    
    print(f"Encoders saved to {encoders_path}")
    return data, label_encoders

# ラベルエンコーディングを適用
encoded_data, encoders = label_encode_categorical_columns(data)
print(encoders)
encoded_data

Encoders saved to model/encoders.joblib
{'categorical_feature_0': LabelEncoder(), 'categorical_feature_1': LabelEncoder()}


,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,label,categorical_feature_0,categorical_feature_1
0,-1.140526,1.359706,0.861991,0.846092,0.606010,-1.556629,1.754794,1.696456,-1.280429,-2.081929,1,2,2
1,-0.078734,-1.329332,0.627375,-1.193006,-0.773010,0.097676,0.497998,0.959271,0.024510,1.451144,1,0,2
2,0.807427,0.730198,-1.285680,0.889484,-1.804882,-0.763259,0.048085,-0.904317,-1.627542,0.259723,0,2,2
3,0.588465,-0.375121,-0.575002,-0.149518,-0.563725,0.412931,0.243687,-0.506943,-0.822220,0.244967,0,2,0
4,1.636312,-1.640607,-1.360456,-0.941163,-1.430141,1.632411,0.130741,-1.435862,-0.440044,1.441273,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.531047,0.948005,-1.032233,0.996309,-1.008086,-2.038125,-0.351513,1.676437,-1.870792,0.018418,0,0,1
96,-0.963142,0.556003,0.964233,0.193754,-0.703176,1.149273,1.770801,1.812449,-0.034988,-0.626967,1,0,2
97,0.950573,-2.255540,-0.270624,-1.692005,0.504047,-0.985726,-0.792873,-1.035242,-0.530258,-0.107030,1,2,2
98,-1.016951,0.554484,1.031102,0.175930,0.615936,-1.081063,-0.309546,-1.251114,0.593101,0.326133,1,0,1


# モデルの訓練

## 訓練/テストデータの分割

In [8]:
def split_train_test(data: pd.DataFrame, test_size: float = 0.2, random_state: int = 42) -> tuple:
    """
    データを訓練データとテストデータに分割する関数。

    :param data: 分割するデータフレーム
    :param test_size: テストデータの割合 (デフォルト: 0.2)
    :param random_state: ランダムシード (デフォルト: 42)
    :return: 訓練データとテストデータのタプル (train, test)
    """
    logger.info(f"Splitting data into train and test sets with test_size={test_size} and random_state={random_state}")
    
    # データの分割
    train, test = train_test_split(data, test_size=test_size, random_state=random_state)
    
    logger.info(f"Data split completed: {len(train)} training samples, {len(test)} test samples")
    
    return train, test

# 使用例
train, test = split_train_test(encoded_data)
print(train.shape)
print(test.shape)

INFO:__main__:Splitting data into train and test sets with test_size=0.2 and random_state=42
INFO:__main__:Data split completed: 80 training samples, 20 test samples


(80, 13)
(20, 13)


## 訓練

In [9]:
def train_lightgbm_model(train_data: pd.DataFrame, label_column: str = 'label', params: dict = None) -> lgb.Booster:
    """
    LightGBMモデルを訓練する関数

    :param train_data: 訓練データを含むDataFrame
    :param label_column: 目的変数の列名 (デフォルト: 'label')
    :param params: LightGBMのハイパーパラメータ (デフォルトはバイナリ分類用)
    :return: 訓練済みLightGBMモデル (lgb.Boosterオブジェクト)
    """
    logger.info('Preparing training data for LightGBM model.')

    # 訓練データとラベルの分割
    X_train = train_data.drop(label_column, axis=1)
    y_train = train_data[label_column]

    logger.info('Training LightGBM model...')
    
    # LightGBM用のデータセットを作成
    train_dataset = lgb.Dataset(X_train, label=y_train)

    # デフォルトのハイパーパラメータを設定（必要に応じて変更可能）
    if params is None:
        params = {
            'objective': 'binary',  # バイナリ分類用
            'metric': 'binary_logloss',  # 評価指標
            'verbosity': -1  # 詳細出力の抑制
        }
    
    # モデルの訓練
    model = lgb.train(params, train_dataset)
    
    logger.info('LightGBM model training completed.')
    
    return model

# 使用例
# 訓練データを分割済みの場合
trained_model = train_lightgbm_model(train)

INFO:__main__:Preparing training data for LightGBM model.
INFO:__main__:Training LightGBM model...
INFO:__main__:LightGBM model training completed.


## モデルの保存&圧縮

In [10]:
def save_and_compress_model(model: lgb.Booster, model_dir: str = 'model', model_filename: str = 'model.joblib', archive_filename: str = 'model.tar.gz'):
    """
    モデルを保存し、ディレクトリ内のファイルを圧縮する関数

    :param model: LightGBM Boosterオブジェクト
    :param model_dir: モデルを保存するディレクトリ (デフォルト: 'model')
    :param model_filename: 保存するモデルファイル名 (デフォルト: 'model.txt')
    :param archive_filename: 圧縮するアーカイブファイル名 (デフォルト: 'model.tar.gz')
    """
    # ディレクトリが存在しない場合は作成
    if not os.path.exists(model_dir):
        os.makedirs(model_dir)
        logger.info(f"Directory '{model_dir}' created.")

    # モデルの保存
    model_path = os.path.join(model_dir, model_filename)
    # model.save_model(model_path)
    joblib.dump(model, model_path)
    logger.info(f"Model saved to {model_path}")

    # モデルディレクトリを圧縮
    archive_path = os.path.join(model_dir, archive_filename)
    with tarfile.open(archive_path, 'w:gz') as tar:
        tar.add(model_dir, arcname='.')
    logger.info(f"Model directory '{model_dir}' compressed to '{archive_path}'")

# 使用例
# 事前に訓練したLightGBMモデルを保存・圧縮する場合
save_and_compress_model(trained_model)

INFO:__main__:Model saved to model/model.joblib
INFO:__main__:Model directory 'model' compressed to 'model/model.tar.gz'


## 圧縮モデルをS3にアップロード

In [11]:
def upload_model_to_s3(model_path: str, s3_prefix: str, session: Session, bucket_name: str) -> str:
    """
    圧縮したモデルファイルをS3にアップロードする関数

    :param model_path: アップロードするモデルファイルのローカルパス
    :param s3_prefix: S3上の保存先プレフィックス
    :param session: SageMakerのセッション
    :param bucket_name: アップロード先のS3バケット名
    :return: アップロードされたモデルファイルのS3 URI
    """
    logger.info(f"Uploading model to S3: {model_path} -> s3://{bucket_name}/{s3_prefix}")
    
    # モデルファイルをS3にアップロード
    model_s3_uri: Final[str] = session.upload_data(
        path=model_path,
        bucket=bucket_name,
        key_prefix=s3_prefix
    )
    
    logger.info(f"Model uploaded to S3 at {model_s3_uri}")
    return model_s3_uri

# 使用例
model_dir = 'model'  # モデルディレクトリ
model_tar_path = f'./{model_dir}/model.tar.gz'

# モデルをS3にアップロード
model_s3_uri = upload_model_to_s3(model_path=model_tar_path, 
                                  s3_prefix=sagemaker_settings["s3_prefix"], 
                                  session=sagemaker_settings["session"], 
                                  bucket_name=sagemaker_settings["bucket"])
print(model_s3_uri)

INFO:__main__:Uploading model to S3: ./model/model.tar.gz -> s3://sagemaker-ap-northeast-1-706711397653/model_deploy_template_multi_dict
INFO:__main__:Model uploaded to S3 at s3://sagemaker-ap-northeast-1-706711397653/model_deploy_template_multi_dict/model.tar.gz


s3://sagemaker-ap-northeast-1-706711397653/model_deploy_template_multi_dict/model.tar.gz


# モデルをデプロイ

In [ ]:
def deploy_model_to_sagemaker(model_s3_uri: str, role: str, entry_point: str, source_dir: str, session: Session,
                              framework: str = "pytorch", version: str = "2.1", py_version: str = "py310",
                              instance_type: str = "ml.m5.large", initial_instance_count: int = 1,
                              endpoint_name: str = "lightgbm-endpoint") -> Model:
    """
    SageMakerにモデルをデプロイする関数

    :param model_s3_uri: S3上のモデルアーティファクトのURI
    :param role: SageMaker用のIAMロール
    :param entry_point: 推論用スクリプトのパス
    :param source_dir: entry_pointを含むディレクトリ
    :param session: SageMakerのセッション
    :param framework: 使用するフレームワーク (デフォルト: 'pytorch')
    :param version: フレームワークのバージョン (デフォルト: '2.1')
    :param py_version: Pythonバージョン (デフォルト: 'py310')
    :param instance_type: デプロイ用のインスタンスタイプ (デフォルト: 'ml.m5.large')
    :param initial_instance_count: デプロイ用のインスタンス数 (デフォルト: 1)
    :param endpoint_name: デプロイするエンドポイントの名前 (デフォルト: 'lightgbm-endpoint')
    :return: デプロイされたモデルのPredictorオブジェクト
    """
    # コンテナURIの取得
    logger.info(f"Retrieving container URI for framework={framework}, version={version}, region={session.boto_region_name}")
    container_uri = retrieve(
        framework=framework,
        region=session.boto_region_name,
        version=version,
        py_version=py_version,
        image_scope="inference",
        instance_type=instance_type,
    )

    # Modelオブジェクトの作成
    logger.info("Creating SageMaker model object...")
    model = Model(
        model_data=model_s3_uri,
        role=role,
        entry_point=entry_point,
        source_dir=source_dir,
        image_uri=container_uri,
        sagemaker_session=session
    )

    # エンドポイントのデプロイ
    logger.info(f"Deploying model to endpoint {endpoint_name} with instance type {instance_type}...")
    predictor = model.deploy(
        initial_instance_count=initial_instance_count,
        instance_type=instance_type,
        endpoint_name=endpoint_name
    )

    logger.info(f"Model deployed successfully at endpoint: {endpoint_name}")
    return predictor

# 使用例
entry_point = 'inference.py'
source_dir = 'source'  # 推論スクリプトと依存ファイルを含むディレクトリ


# 現在の年月日時分を文字列にする
current_datetime_str = datetime.now().strftime("%Y%m%d%H%M")
endpoint_name = "model-"+current_datetime_str
predictor = deploy_model_to_sagemaker(model_s3_uri=model_s3_uri,
                                      role = sagemaker_settings["role"],
                                      entry_point = entry_point,
                                      source_dir = source_dir,
                                      session = sagemaker_settings["session"],
                                      endpoint_name = endpoint_name)

INFO:__main__:Retrieving container URI for framework=pytorch, version=2.1, region=ap-northeast-1
INFO:__main__:Creating SageMaker model object...
INFO:__main__:Deploying model to endpoint model-202411041451 with instance type ml.m5.large...
INFO:sagemaker:Repacking model artifact (s3://sagemaker-ap-northeast-1-706711397653/model_deploy_template_multi_dict/model.tar.gz), script artifact (source), and dependencies ([]) into single tar.gz file located at s3://sagemaker-ap-northeast-1-706711397653/pytorch-inference-2024-11-04-14-51-12-331/model.tar.gz. This may take some time depending on model size...
INFO:sagemaker:Creating model with name: pytorch-inference-2024-11-04-14-51-12-570
INFO:sagemaker:Creating endpoint-config with name model-202411041451
INFO:sagemaker:Creating endpoint with name model-202411041451


------

# 推論テスト

In [69]:
def test_endpoint_prediction(endpoint_name: str, input_data: dict, session, serializer=JSONSerializer(), deserializer=JSONDeserializer()) -> dict:
    """
    デプロイされたエンドポイントに対して推論を実行する関数

    :param endpoint_name: デプロイされたエンドポイントの名前
    :param input_data: 推論に使用するデータ
    :param session: SageMakerセッション
    :param serializer: 入力データのシリアライザ (デフォルト: JSONSerializer)
    :param deserializer: 出力データのデシリアライザ (デフォルト: JSONDeserializer)
    :return: エンドポイントからの推論結果
    """
    logger.info(f"Creating Predictor object for endpoint '{endpoint_name}'")

    # Predictorオブジェクトの作成
    predictor = Predictor(
        endpoint_name=endpoint_name,
        sagemaker_session=session,
        serializer=serializer,
        deserializer=deserializer
    )

    logger.info("Sending data to the endpoint for prediction")
    
    # 推論の実行
    response = predictor.predict(input_data)
    
    logger.info("Prediction completed")
    return response

### 使用例
# 複数dataの例
input_data = {
    "data1": [{}],
    "data2": [{}],
    # "data3":[{}]
}
value_list = [1, 10]

for i in range(0, 10):
    feature_name = f"feature_{i}"
    input_data["data1"][0][feature_name] = random.choice([0, 1])
    input_data["data2"][0][feature_name] = random.choice(value_list)

input_data["data1"][0]["categorical_feature_0"] = "Category_1"
input_data["data1"][0]["categorical_feature_1"] = "Category_2"
input_data["data2"][0]["categorical_feature_0"] = "Category_1"
input_data["data2"][0]["categorical_feature_1"] = "Category_2"
# print(input_data)

# エンドポイントでの推論実行
session = sagemaker_settings["session"]  # 事前に初期化したSageMaker設定を使用
prediction_result = test_endpoint_prediction(endpoint_name = endpoint_name,
                                             input_data = input_data,
                                             session = session)

# 結果の表示
print(prediction_result)

INFO:__main__:Creating Predictor object for endpoint 'model-202411041439'
INFO:__main__:Sending data to the endpoint for prediction
INFO:__main__:Prediction completed


[{'feature_0': 1, 'feature_1': 1, 'feature_2': 1, 'feature_3': 0, 'feature_4': 1, 'feature_5': 1, 'feature_6': 0, 'feature_7': 0, 'feature_8': 0, 'feature_9': 1, 'categorical_feature_0': 'Category_1', 'categorical_feature_1': 'Category_2', 'prediction': 0.9953916896}]


# テスト

## inference.py

### model_fn()

In [78]:
# モジュールの再読み込み
import source.inference
importlib.reload(source.inference)
from source.inference import model_fn

### モデルディレクトリのパス
model_dir = './model'

### モデルのロード
model, label_encoders = model_fn(model_dir)

### モデルが正しくロードされたか確認
print("モデルがロードされました:", model is not None)

INFO:source.inference:推論モデルをロードしています。
INFO:source.inference:推論モデルをロード完了。
INFO:source.inference:label encoderモデルをロードしています。
INFO:source.inference:label encoderモデルをロード完了。


モデルがロードされました: True


### input_fn()

In [79]:
# モジュールの再読み込み
import source.inference
importlib.reload(source.inference)
from source.inference import input_fn

### テスト用のリクエストボディ（JSON形式）
test_request_body  =json.dumps({'data1': [{'feature_0': 0,
   'feature_1': 0,
   'feature_2': 1,
   'feature_3': 0,
   'feature_4': 1,
   'feature_5': 0,
   'feature_6': 1,
   'feature_7': 0,
   'feature_8': 1,
   'feature_9': 1,
   'categorical_feature_0': "Category_1",
   'categorical_feature_1': "Category_2"}],
 'data2': [{'feature_0': 10,
   'feature_1': 10,
   'feature_2': 1,
   'feature_3': 1,
   'feature_4': 10,
   'feature_5': 1,
   'feature_6': 10,
   'feature_7': 1,
   'feature_8': 1,
   'feature_9': 1,
   'categorical_feature_0': 0,
   'categorical_feature_1': 0}]})

request_content_type = 'application/json'

### 入力データの処理
input_data = input_fn(test_request_body, request_content_type)

### 処理結果の確認
# print("入力データ:", input_data)
input_data

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,categorical_feature_0,categorical_feature_1
0,0,1000,1,0,1,0,1,0,1,1,Category_1,Category_2


### predecit_fn()

In [80]:
# モジュールの再読み込み
import source.inference
importlib.reload(source.inference)

from source.inference import predict_fn

# モデルと入力データを使用して予測
prediction = predict_fn(input_data, model_fn(model_dir))

# 予測結果の確認
print("予測結果:", prediction)

INFO:source.inference:推論モデルをロードしています。
INFO:source.inference:推論モデルをロード完了。
INFO:source.inference:label encoderモデルをロードしています。
INFO:source.inference:label encoderモデルをロード完了。


予測結果: (   feature_0  feature_1  feature_2  feature_3  feature_4  feature_5  \
0          0       1000          1          0          1          0   

   feature_6  feature_7  feature_8  feature_9  categorical_feature_0  \
0          1          0          1          1                      1   

   categorical_feature_1  prediction  
0                      2    0.997288  , {'categorical_feature_0': LabelEncoder(), 'categorical_feature_1': LabelEncoder()})


### output_fn()

In [81]:
# モジュールの再読み込み
import source.inference
importlib.reload(source.inference)
from source.inference import output_fn

response_content_type = 'application/json'

### 出力データの生成
response = output_fn(prediction, response_content_type)

### 応答の確認
print(response)

INFO:source.inference:Data saved to s3://model_deploy_template_multi_dict/output_from_modelendpoint/output.csv


[{"feature_0":0,"feature_1":1000,"feature_2":1,"feature_3":0,"feature_4":1,"feature_5":0,"feature_6":1,"feature_7":0,"feature_8":1,"feature_9":1,"categorical_feature_0":"Category_1","categorical_feature_1":"Category_2","prediction":0.9972881481}]


In [18]:
prediction

array([0.9353699])